In [2]:
import os
from utils import *
import optuna
from Recommenders.Recommender_import_list import *

from sklearn.model_selection import KFold
from scipy import sparse as sps

In [10]:
from itertools import chain

data, usermap, itemmap, users = load_data2()
data_train, data_val = split_data2(data, 0.2)
data_train_val = data_train + data_val

k = 5
kf = KFold(n_splits=k, shuffle=True, random_state=42)
folds = list(kf.split(data_train_val))

candidate_recommender_study = optuna.create_study(
    study_name="P3Beta",
    #storage=get_database_url(),
    load_if_exists=True,
    direction="maximize",
)
candidate_recommender = RP3betaRecommender(data_train_val, verbose=False)
candidate_recommender.fit()

n_users, n_items = data_train_val.shape

training_dataframe = pd.DataFrame(index=range(0,n_users), columns = ["ItemID"])
training_dataframe.index.name='UserID'

cutoff = 50

for user_id in range(n_users):    
    recommendations = candidate_recommender.recommend(user_id, cutoff = cutoff, remove_seen_flag=True)
    training_dataframe.loc[user_id, "ItemID"] = recommendations
training_dataframe = training_dataframe.explode("ItemID")

for fold, (train_index, val_index) in enumerate(folds):
    # Dividi il dataset in training e validation
    train_fold, val_fold = data_val[train_index], data_val[val_index]

    # Valuta il modello su val_fold
    predictions = []

    for user_id in range(val_fold.shape[0]):
        recommendations = candidate_recommender.recommend(user_id, cutoff=cutoff, remove_seen_flag=True)
        predictions.append(recommendations)


    # Flatten predictions e val_fold per calcolare le etichette
    predictions_flat = list(chain.from_iterable(predictions))
    val_fold_coo = sps.coo_matrix(val_fold)
    correct_recommendations = pd.DataFrame({"UserID": val_fold_coo.row, "ItemID": val_fold_coo.col})
    
    # Calcola le etichette sulla base della corrispondenza tra le previsioni e val_fold
    labels = [(item,) in correct_recommendations.itertuples(index=False, name=None) for item in predictions_flat]

    # Aggiungi le etichette al dataframe
    training_dataframe.loc[val_index, f"Label_fold_{fold}"] = labels

[I 2024-01-01 18:34:54,928] A new study created in memory with name: P3Beta


KeyboardInterrupt: 